In [ ]:
import pandas as pd 

In [ ]:
data = pd.read_csv("sudoku_cluewise/sudoku_cluewise.csv")

In [ ]:
# Grouping sudoku on basis of difficulty

def determine_difficulty(clue_numbers):
    
    if clue_numbers >= 50:
        return "easy"
    elif clue_numbers >= 32:
        return "medium"
    elif clue_numbers >= 28:
        return "hard"
    else:
        return "very_hard"

In [ ]:
data["difficulty_level"] = data['clue_numbers'].apply(determine_difficulty)

In [ ]:
# Selecting only 10 sudokus of each difficulty level
selected_sudokus = pd.DataFrame()
for difficulty in data['difficulty_level'].unique():
    subset = data[data['difficulty_level'] == difficulty].head(10)
    selected_sudokus = pd.concat([selected_sudokus, subset])

In [ ]:
selected_sudokus.to_csv("selected_sudokus.csv", index=False)

In [ ]:
#data.to_csv("sudoku_cluewise.csv")

In [1]:
import pandas as pd
import numpy as np
import time
from collections import defaultdict
import random
from collections import deque

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [71]:
def read_sudoku(path):
    df = pd.read_csv(path)  
    puzzles = df['quizzes'].tolist()
    solutions = df['solutions'].tolist()
    difficulties = df['difficulty_level'].tolist()
    return puzzles, solutions, difficulties

In [24]:
def parse_sudoku(puzzle):
    grid = np.array([int(x) if x.isdigit() else 0 for x in puzzle]).reshape((9, 9))
    return grid

In [59]:
def parse_sudoku_solution(solution):
    if isinstance(solution, int):
        # If the solution is a single integer, convert it into a 9x9 grid format
        puzzle_str = str(solution).zfill(81)
        grid = np.array([int(x) for x in puzzle_str]).reshape((9, 9))
    else:
        # If the solution is already in a grid format, parse it directly
        if isinstance(solution, str):
            # Convert string representation to list of integers
            solution = [int(x) for x in solution]
        grid = np.array(solution).reshape((9, 9))
    return grid

In [25]:
def possible_values(grid, r, c):
    if grid[r, c] != 0:
        return set()
    possible = set(range(1, 10))  
    possible -= set(grid[r, :]) 
    possible -= set(grid[:, c]) 
    block_row, block_col = 3 * (r // 3), 3 * (c // 3)
    possible -= set(grid[block_row:block_row+3, block_col:block_col+3].flatten())  #
    return possible

In [26]:
def forward_checking(grid, r, c, value):
    grid[r, c] = value  # Temporarily place the value
    for i in range(9):
        if i != c and grid[r, i] == 0 and not possible_values(grid, r, i):
            grid[r, c] = 0  # Reset
            return False
        if i != r and grid[i, c] == 0 and not possible_values(grid, i, c):
            grid[r, c] = 0  # Reset
            return False
    grid[r, c] = 0  # Reset
    return True

In [27]:
def hill_climbing_solver(grid):
    progress = True
    while progress:
        progress = False
        for r in range(9):
            for c in range(9):
                if grid[r, c] == 0:
                    possible = possible_values(grid, r, c)
                    for value in possible:
                        grid[r, c] = value
                        progress = True
                        break
    return grid

In [28]:
def hill_climbing_with_forward_checking(grid):
    progress = True
    while progress:
        progress = False
        for r in range(9):
            for c in range(9):
                if grid[r, c] == 0:
                    possible = possible_values(grid, r, c)
                    for value in possible:
                        if forward_checking(grid, r, c, value):
                            grid[r, c] = value
                            progress = True
                            break
    return grid


In [29]:
def hill_climbing_with_arc_consistency(grid):
    if not arc_consistency(grid):
        return False
    progress = True
    while progress:
        progress = False
        for r in range(9):
            for c in range(9):
                if grid[r, c] == 0:
                    possible = possible_values(grid, r, c)
                    for value in possible:
                        grid[r, c] = value
                        if arc_consistency(grid):
                            progress = True
                            break
                        else:
                            grid[r, c] = 0
                    if progress:
                        break
    return grid


In [98]:
def is_valid_move(grid, row, col, num):
    if num in grid[row]:
        return False
    
    for i in range(9):
        if grid[i][col] == num:
            return False
    
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if grid[start_row + i][start_col + j] == num:
                return False
    
    return True

In [99]:
def find_empty_location(grid):
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                return i, j
    return -1, -1

In [100]:
def solve_sudoku_backtracking(grid):
    row, col = find_empty_location(grid)
    if row == -1 and col == -1:
        return True
    
    for num in range(1, 10):
        if is_valid_move(grid, row, col, num):
            grid[row][col] = num
            
            if solve_sudoku_backtracking(grid):
                return True
            
            grid[row][col] = 0
    
    return False

In [101]:
def solve_sudoku_with_backtrack_and_forward_checking(grid):
    def solve_backtrack(grid):
        empty = find_empty_location(grid)
        if not empty:
            return True  
        r, c = empty
        for num in range(1, 10):
            if is_valid_move(grid, r, c, num):
                if forward_checking(grid, r, c, num):
                    grid[r][c] = num
                    if solve_backtrack(grid):
                        return True
                    grid[r][c] = 0  
        return False

    time_taken = []
    for _ in range(10):  
        start_time = time.time() * 1000
        solve_backtrack(grid.copy())
        end_time = time.time() * 1000
        time_taken.append(end_time - start_time)
    return sum(time_taken) / len(time_taken)

In [102]:
def revise(grid, i, j, value):
    removed = False
    for x in range(9):
        if x != j and value in grid[i][x]:
            grid[i][x].remove(value)
            removed = True
        if x != i and value in grid[x][j]:
            grid[x][j].remove(value)
            removed = True
    start_row, start_col = 3 * (i // 3), 3 * (j // 3)
    for x in range(start_row, start_row + 3):
        for y in range(start_col, start_col + 3):
            if x != i and y != j and value in grid[x][y]:
                grid[x][y].remove(value)
                removed = True
    return removed

In [103]:
def arc_consistency(grid):
    
    queue = [(i, j) for i in range(9) for j in range(9) if isinstance(grid[i][j], list)] 
    while queue:
        i, j = queue.pop(0)
        if grid[i][j]: 
            value = grid[i][j][0]
            if revise(grid, i, j, value):
                if not grid[i][j]:  
                    return False
                queue.extend((x, y) for x in range(9) for y in range(9) if (x != i or y != j) and isinstance(grid[x][y], list) and (x == i or y == j or (x // 3 == i // 3 and y // 3 == j // 3)))  # Add neighbors to the queue
    return True

In [104]:
def solve_sudoku_backtracking_arc_consistency(grid):
    row, col = find_empty_location(grid)
    if row == -1 and col == -1:
        return True

    # Checking if the cell already has a fixed value
    if isinstance(grid[row][col], int):
        return solve_sudoku_backtracking_arc_consistency(grid)

    # Checking if the cell has any values in its domain
    if not grid[row][col]:
        return False

    for num in grid[row][col]:
        grid_copy = copy.deepcopy(grid)
        grid_copy[row][col] = [num]

        if is_valid_move(grid_copy, row, col, num):
            if solve_sudoku_backtracking_arc_consistency(grid_copy):
                for i in range(9):
                    for j in range(9):
                        if isinstance(grid_copy[i][j], list) and len(grid_copy[i][j]) == 1:
                            grid[i][j] = grid_copy[i][j][0]
                return True

    return False


In [105]:
class SudokuSolverQL:
    def __init__(self, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        # Dictionary to store Q-values
        self.q_values = {}  
        
    def get_q_value(self, state, action):
        return self.q_values.get((state, action), 0.0)

    def update_q_value(self, state, action, reward, next_state):
        max_next_q_value = max([self.get_q_value(next_state, next_action) for next_action in range(1, 10)])
        current_q_value = self.get_q_value(state, action)
        new_q_value = current_q_value + self.learning_rate * (reward + self.discount_factor * max_next_q_value - current_q_value)
        self.q_values[(state, action)] = new_q_value

    def select_action(self, state, valid_actions):
        if random.random() < self.exploration_rate:
            return random.choice(valid_actions)
        else:
            max_q_value = max([self.get_q_value(state, action) for action in valid_actions])
            best_actions = [action for action in valid_actions if self.get_q_value(state, action) == max_q_value]
            return random.choice(best_actions)

    def solve_sudoku(self, puzzle):
        puzzle_str = [str(row) for row in puzzle]
        
        grid = np.array([[int(char) if char.isdigit() else 0 for char in row] for row in puzzle_str])
    
        # Initialize Q-learning agent
        agent = SudokuSolverQL()
    
        num_episodes = 10000
        for episode in range(num_episodes):
            state = tuple(map(tuple, grid))
            while not self.is_solved(grid):
                valid_actions = self.get_valid_actions(grid)
                action = agent.select_action(state, valid_actions)
                next_grid = self.take_action(grid, action)
                reward = self.calculate_reward(grid, next_grid)
                next_state = tuple(map(tuple, next_grid))
                agent.update_q_value(state, action, reward, next_state)
                grid = next_grid
                state = next_state
    
        return grid

    def is_solved(self, grid):
        return np.all(grid != 0)

    def get_valid_actions(self, grid):
        empty_cells = [(i, j) for i in range(grid.shape[0]) for j in range(grid.shape[1]) if grid[i][j] == 0]
        return [(i, j, num) for i, j in empty_cells for num in range(1, 10)]

    def take_action(self, grid, action):
        i, j, num = action
        next_grid = grid.copy()
        next_grid[i][j] = num
        return next_grid

    def calculate_reward(self, current_grid, next_grid):
        if self.is_solved(next_grid):
            return 1000
        elif np.count_nonzero(next_grid) > np.count_nonzero(current_grid):
            return 1
        else:
            return 0

In [107]:
def main():
    
    algorithms = {
        "Hill Climbing": hill_climbing_solver,
        "Hill Climbing with Forward Checking": hill_climbing_with_forward_checking,
        "Hill Climbing with Arc Consistency": hill_climbing_with_arc_consistency,
        "Backtracking": solve_sudoku_backtracking,
        "Backtracking with Forward Checking": solve_sudoku_with_backtrack_and_forward_checking,
        #"Backtracking with Arc Consistency": solve_sudoku_backtracking_arc_consistency,
        "Q-learning": SudokuSolverQL().solve_sudoku
    }

    puzzles,solutions, difficulties = read_sudoku('selected_sudokus.csv')
    time_dict = {alg: defaultdict(list) for alg in algorithms.keys()}
    accuracy_dict = {alg: [] for alg in algorithms.keys()}

    for puzzle, solution, difficulty in zip(puzzles, solutions, difficulties):
        grid = parse_sudoku(puzzle)
        actual_solution = parse_sudoku_solution(solution)
        for alg_name, solver_func in algorithms.items():
            solved_grid = solver_func(grid.copy())
            accuracy = calculate_accuracy(solved_grid, actual_solution)
            accuracy_dict[alg_name].append((difficulty, accuracy))

            start_time = time.time() * 1000
            solver_func(grid.copy())
            end_time = time.time() * 1000
            solve_time = end_time - start_time
            time_dict[alg_name][difficulty].append(solve_time)

    # Printing the average solve times for each algorithm and difficulty level
    for alg_name, alg_times in time_dict.items():
        print(f"Average solve time for {alg_name}:")
        for difficulty, times in alg_times.items():
            average_time = sum(times) / len(times)
            print(f"Difficulty {difficulty}: {average_time:.2f} milliseconds")
        print()  
        
if __name__ == "__main__":
    main_results = main()

C:\Users\athir\AppData\Local\Temp\ipykernel_37072\3402303637.py:41: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  correct_cells = np.sum(solved_grid == actual_solution)


Average solve time for Hill Climbing:
Difficulty easy: 0.05 milliseconds
Difficulty medium: 1.06 milliseconds
Difficulty hard: 1.87 milliseconds
Difficulty very_hard: 1.87 milliseconds

Average solve time for Hill Climbing with Forward Checking:
Difficulty easy: 0.00 milliseconds
Difficulty medium: 2.06 milliseconds
Difficulty hard: 8.32 milliseconds
Difficulty very_hard: 8.17 milliseconds

Average solve time for Hill Climbing with Arc Consistency:
Difficulty easy: 0.22 milliseconds
Difficulty medium: 4.50 milliseconds
Difficulty hard: 8.47 milliseconds
Difficulty very_hard: 8.73 milliseconds

Average solve time for Backtracking:
Difficulty easy: 0.00 milliseconds
Difficulty medium: 5.05 milliseconds
Difficulty hard: 140.23 milliseconds
Difficulty very_hard: 9135.24 milliseconds

Average solve time for Backtracking with Forward Checking:
Difficulty easy: 0.86 milliseconds
Difficulty medium: 84.71 milliseconds
Difficulty hard: 20102.37 milliseconds
Difficulty very_hard: 356713.40 millis